In [5]:
#!/usr/bin/env python3
""" Redacting Formatter class
"""
import os
import re
from typing import List
import logging
import mysql.connector


PII_FIELDS = ("name", "email", "phone", "address", "ssn")

# CLASSES
""" Redacting Formatter class
"""


class RedactingFormatter(logging.Formatter):
    """ Redacting Formatter class
    """

    REDACTION = "***"
    FORMAT = "[HOLBERTON] %(name)s %(levelname)s %(asctime)-15s: %(message)s"
    SEPARATOR = "; "

    def __init__(self, fields: list):
        super(RedactingFormatter, self).__init__(self.FORMAT)
        self.fields = fields

    def format(self, record: logging.LogRecord) -> str:
        record.msg = filter_datum(self.fields, self.REDACTION, record.msg,
                                  self.SEPARATOR)
        return super().format(record)

# METHODS


"""returns the log message obfuscated"""


def filter_datum(fields: List[str], redaction: str, message: str,
                 separator: str) -> str:
    """returns the log message obfuscated"""
    p = fr'(\b|{re.escape(separator)})({"|".join(map(re.escape, fields))})=([^;]+)'
    return re.sub(p, lambda match: f'{match.group(1)}{match.group(2)}={redaction}', message)


def get_logger() -> logging.Logger:
    """Return a configured logger"""
    logger = logging.getLogger("user_data")
    logger.setLevel(logging.INFO)

    formatter = RedactingFormatter(fields=PII_FIELDS)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)

    logger.addHandler(stream_handler)
    logger.propagate = False

    return logger

In [6]:
"""
Main file
"""

import logging
import re

fields = ["password", "date_of_birth"]
messages = ["name=egg;email=eggmin@eggsample.com;password=eggcellent;date_of_birth=12/12/1986;", "name=bob;email=bob@dylan.com;password=bobbycool;date_of_birth=03/04/1993;"]

for message in messages:
    print(filter_datum(fields, 'xxx', message, ';'))
print(filter_datum.__annotations__.get('return'))

print('\n====================================================================\n')

print(RedactingFormatter.format.__annotations__.get('return'))
message = "name=Bob;email=bob@dylan.com;ssn=000-123-0000;password=bobby2019;"
log_record = logging.LogRecord("my_logger", logging.INFO, None, None, message, None, None)
formatter = RedactingFormatter(fields=("email", "ssn", "password"))
print(formatter.format(log_record))

print('\n====================================================================\n')

print(get_logger.__annotations__.get('return'))
print("PII_FIELDS: {}".format(len(PII_FIELDS)))


print('\n====================================================================\n')

name=egg;email=eggmin@eggsample.com;password=xxx;date_of_birth=xxx;
name=bob;email=bob@dylan.com;password=xxx;date_of_birth=xxx;
<class 'str'>


<class 'str'>
[HOLBERTON] my_logger INFO 2024-02-09 01:58:25,081: name=Bob;email=***;ssn=***;password=***;


<class 'logging.Logger'>
PII_FIELDS: 5


